In [1]:
!pip install optuna
!pip install xgboost
!pip install imblearn
import pandas as pd
import numpy as np
import optuna
import xgboost as xgb
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

class CropPredictor:
    def __init__(self):
        self.label_encoder_soil = LabelEncoder()
        self.label_encoder_period = LabelEncoder()
        self.label_encoder_crop = LabelEncoder()
        self.scaler = StandardScaler()
        self.model = None
        self.feature_names = None

    def preprocess_data(self, data, is_training=True):
        # Create a copy of the data
        processed_data = data.copy()

        if is_training:
            # Fit and transform for training data
            processed_data['Soil Type'] = self.label_encoder_soil.fit_transform(processed_data['Soil Type'])
            processed_data['Period of Month'] = self.label_encoder_period.fit_transform(processed_data['Period of Month'])
        else:
            # Transform only for prediction data
            processed_data['Soil Type'] = self.label_encoder_soil.transform(processed_data['Soil Type'])
            processed_data['Period of Month'] = self.label_encoder_period.transform(processed_data['Period of Month'])

        # Feature Engineering
        processed_data['NPK_Ratio'] = processed_data['Nitrogen'] / (processed_data['Phosphorus'] + processed_data['Potassium'])
        processed_data['Temp_Humidity_Index'] = processed_data['Temperature'] * processed_data['Humidity'] / 100
        processed_data['Soil_Moisture_Index'] = np.log1p(processed_data['Temperature'] * processed_data['Humidity'])
        processed_data['NPK_TH_Index'] = processed_data['NPK_Ratio'] * processed_data['Temp_Humidity_Index']

        return processed_data

    def objective(self, trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 500, step=50),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
            'subsample': trial.suggest_float('subsample', 0.6, 0.9),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
            'gamma': trial.suggest_float('gamma', 0.01, 0.5, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 1.0, log=True),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 1.0, log=True)
        }

        model = xgb.XGBClassifier(**params, objective='multi:softprob', eval_metric='mlogloss', random_state=42)

        cv_scores = []
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

        for train_idx, val_idx in cv.split(self.X_train, self.y_train):
            X_fold_train, X_fold_val = self.X_train[train_idx], self.X_train[val_idx]
            y_fold_train, y_fold_val = self.y_train[train_idx], self.y_train[val_idx]

            model.fit(X_fold_train, y_fold_train)
            preds = model.predict(X_fold_val)
            cv_scores.append(accuracy_score(y_fold_val, preds))

        return np.mean(cv_scores)

    def train(self, data_path):
        print("Loading and preprocessing data...")
        data = pd.read_csv(data_path)

        # Encode target variable
        y = self.label_encoder_crop.fit_transform(data['Crop'])

        # Preprocess features
        X = self.preprocess_data(data.drop('Crop', axis=1), is_training=True)
        self.feature_names = X.columns.tolist()

        # Convert to numpy arrays
        X = X.values

        print("Applying SMOTE for class balancing...")
        smote = SMOTE(sampling_strategy='auto', random_state=42)
        X_resampled, y_resampled = smote.fit_resample(X, y)

        self.X_train, X_test, self.y_train, y_test = train_test_split(
            X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42
        )

        # Scale features
        self.X_train = self.scaler.fit_transform(self.X_train)
        X_test = self.scaler.transform(X_test)

        print("Optimizing hyperparameters...")
        study = optuna.create_study(direction='maximize')
        study.optimize(self.objective, n_trials=30, timeout=600)

        best_params = study.best_params
        print(f"Best parameters: {best_params}")
        print(f"Best CV accuracy: {study.best_value:.4f}")

        print("Training final model...")
        self.model = xgb.XGBClassifier(
            **best_params,
            objective='multi:softprob',
            eval_metric='mlogloss',
            random_state=42
        )
        self.model.fit(self.X_train, self.y_train)

        preds = self.model.predict(X_test)
        accuracy = accuracy_score(y_test, preds)
        print(f"\nFinal Model Accuracy: {accuracy:.4f}")
        print("\nClassification Report:")
        print(classification_report(y_test, preds))

        print("Saving model and preprocessors...")
        with open('crop_prediction_model.pkl', 'wb') as f:
            pickle.dump({
                'model': self.model,
                'label_encoder_crop': self.label_encoder_crop,
                'label_encoder_soil': self.label_encoder_soil,
                'label_encoder_period': self.label_encoder_period,
                'scaler': self.scaler,
                'feature_names': self.feature_names
            }, f)

        return accuracy

    def predict_crop(self, input_data):
        # Convert input to DataFrame if it's a dictionary
        if isinstance(input_data, dict):
            input_data = pd.DataFrame([input_data])

        # Preprocess the input data
        processed_data = self.preprocess_data(input_data, is_training=False)

        # Ensure column order matches training data
        processed_data = processed_data[self.feature_names]

        # Scale the features
        scaled_data = self.scaler.transform(processed_data)

        # Make prediction
        prediction = self.model.predict(scaled_data)
        probabilities = self.model.predict_proba(scaled_data)

        predicted_crop = self.label_encoder_crop.inverse_transform(prediction)
        return predicted_crop[0], probabilities[0]

def main():
    predictor = CropPredictor()
    accuracy = predictor.train('augmented_smart_crop_data.csv')

    # Sample inputs
    sample_inputs = [
        {
            'Nitrogen': 90,
            'Phosphorus': 42,
            'Potassium': 43,
            'Temperature': 25,
            'Humidity': 82,
            'pH': 6.5,
            'Soil Type': 'Clayey',
            'Period of Month': 'Kharif'
        },
        {
            'Nitrogen': 120,
            'Phosphorus': 35,
            'Potassium': 30,
            'Temperature': 28,
            'Humidity': 75,
            'pH': 7.0,
            'Soil Type': 'Loamy',
            'Period of Month': 'Rabi'
        }
    ]

    print("\nMaking predictions for sample inputs:")
    for i, sample in enumerate(sample_inputs, 1):
        predicted_crop, probabilities = predictor.predict_crop(sample)
        print(f"\nSample {i}:")
        print(f"Input Parameters: {sample}")
        print(f"Predicted Crop: {predicted_crop}")
        print(f"Confidence: {max(probabilities)*100:.2f}%")

        top_3_indices = np.argsort(probabilities)[-3:][::-1]
        print("\nTop 3 Crop Recommendations:")
        for idx in top_3_indices:
            crop_name = predictor.label_encoder_crop.inverse_transform([idx])[0]
            print(f"{crop_name}: {probabilities[idx]*100:.2f}%")

if __name__ == "__main__":
    main()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.7 MB/s eta 0:00:00


[I 2025-02-25 23:58:51,778] A new study created in memory with name: no-name-b45f7aea-e9db-4fe3-9feb-860bdb3722a6


Loading and preprocessing data...
Applying SMOTE for class balancing...
Optimizing hyperparameters...


[I 2025-02-25 23:59:29,022] Trial 0 finished with value: 0.6337074386866919 and parameters: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.0902966227631913, 'subsample': 0.7290081745672452, 'colsample_bytree': 0.8531157258605591, 'gamma': 0.10924260124889326, 'reg_lambda': 0.09587822027128405, 'reg_alpha': 0.01979499013941734}. Best is trial 0 with value: 0.6337074386866919.
[I 2025-02-25 23:59:56,221] Trial 1 finished with value: 0.4513841322969954 and parameters: {'n_estimators': 250, 'max_depth': 3, 'learning_rate': 0.03671461852558892, 'subsample': 0.6706225994420606, 'colsample_bytree': 0.8351465221712647, 'gamma': 0.026839576163925974, 'reg_lambda': 0.39143615689137384, 'reg_alpha': 0.004273472603650687}. Best is trial 0 with value: 0.6337074386866919.
[I 2025-02-26 00:01:21,937] Trial 2 finished with value: 0.5888458519163914 and parameters: {'n_estimators': 500, 'max_depth': 5, 'learning_rate': 0.0334654085048001, 'subsample': 0.6609694858565651, 'colsample_bytree': 0

Best parameters: {'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.04477880981416993, 'subsample': 0.8515827698858095, 'colsample_bytree': 0.7820311199831482, 'gamma': 0.011427377339104731, 'reg_lambda': 0.0030781068111066807, 'reg_alpha': 0.001049083225551397}
Best CV accuracy: 0.6653
Training final model...

Final Model Accuracy: 0.7591

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.86      0.86        14
           1       0.67      0.86      0.75        14
           2       0.73      0.79      0.76        14
           3       0.76      0.93      0.84        14
           4       0.68      0.93      0.79        14
           5       0.75      0.86      0.80        14
           6       0.91      0.71      0.80        14
           7       0.60      0.86      0.71        14
           8       0.71      0.86      0.77        14
           9       0.69      0.79      0.73        14
          10       0.86      0.8

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
